# Code

In [128]:
import json
import re
import os
import pandas as pd

from typing import Union, List

def firestore_to_json(value_dict):
    if not isinstance(value_dict, dict):
        return value_dict

    for key, value in value_dict.items():
        if key == "stringValue":
            return value
        elif key == "integerValue":
            return int(value)
        elif key == "doubleValue":
            return float(value)
        elif key == "booleanValue":
            return value
        elif key == "nullValue":
            return None
        elif key == "timestampValue":
            return value
        elif key == "mapValue":
            if "fields" in value:
                 return {f_key: firestore_to_json(f_val) for f_key, f_val in value["fields"].items()}
            else:
                 return {}
        elif key == "arrayValue":
            if "values" in value:
                return [firestore_to_json(item) for item in value["values"]]
            else:
                return []
        else:
            return {key: value}
    return None


def rename_df(df : pd.DataFrame) -> pd.DataFrame:
    columns = df.columns.tolist()
    if "price_out" in columns:
        df = df.rename(columns={'price_out': 'Price'})
    if "format" in columns:
        df = df.rename(columns={'format': 'Volume'})
    for c in columns:
        df = df.rename(columns={c: c.capitalize()})
    return df


def display_top(df: pd.DataFrame, n=20) -> pd.DataFrame:
    out = df.copy()
    out = out.reset_index(drop=True)
    out.index.name = "Rank"
    out.index = out.index + 1
    return out.head(n)


def parse(filename: str, columns_to_keep: List[str]) -> Union[pd.DataFrame, None]:
    with open(filename, 'r', encoding="utf-8") as file:
        content = file.read()
    
    match = re.search(r'\[2,\s*\[\s*(\{.*?\})\s*\]\s*\]', content, re.DOTALL)

    if match:
        json_part_string = match.group(1)
        try:
            data = json.loads(json_part_string)

            articles_firestore_map = data.get("documentChange", {}) \
                                        .get("document", {}) \
                                        .get("fields", {}) \
                                        .get("articles", {}) \
                                        .get("mapValue", {}) \
                                        .get("fields", {})

            simplified_articles = {}

            for article_id, article_data in articles_firestore_map.items():
                if "mapValue" in article_data and "fields" in article_data["mapValue"]:
                    simplified_article_fields = {}
                    for field_name, field_value_dict in article_data["mapValue"]["fields"].items():
                        simplified_article_fields[field_name] = firestore_to_json(field_value_dict)
                    simplified_articles[article_id] = simplified_article_fields

            final_json_output = json.dumps(simplified_articles, indent=2, ensure_ascii=False)
            df = pd.DataFrame(simplified_articles).T
            df = df[df['article_type'] == 1]
            df = df.reindex(columns=columns_to_keep)
            df = rename_df(df)
            df = df.reset_index(drop=True)
            with open(f"db/{filename.split('.')[0].split('/')[-1]}.json", "w", encoding="utf-8") as out:
                out.write(final_json_output)

            return df

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON part: {e}")
        except KeyError as e:
            print(f"Error navigating the data structure, missing key: {e}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

    else:
        print("Could not find the relevant data structure '[2, [{...}]]' in the input string.")


def create_dir_if_absent(dir: str) -> None:
    if not os.path.exists(dir):
        os.makedirs(dir)


YEAR = "2025-2026-Q1"
df = parse(f"db/raw/{YEAR}.txt", ['name', 'price_out', 'format', 'type', 'degree', 'available'])

df = df[df["Available"] == True]
df["Ratio"] = (df['Degree'] * df['Volume']) / df['Price']

df = df.sort_values(by=["Ratio", "Price", "Volume"], ascending=[False, True, False])
df = df.reset_index(drop=True)
df.index.name = "Rank"
df.index = df.index + 1

create_dir_if_absent(f"csv/{YEAR}")
df.to_csv(f"csv/{YEAR}/ranker.csv")

# Top 20, any type of beer

In [129]:
display_top(df[df["Volume"] < 75])

,Name,Price,Volume,Type,Degree,Available,Ratio
Rank,,,,,,,
1,Kerel Kaishaku Fût,2.5,25,Blonde,15,True,150.0
2,Kerel Kaishaku,3.5,33,Blonde,15,True,141.428571
3,Kasteel triple,2.6,33,Blonde,11,True,139.615385
4,Bush 10,2.5,33,Blonde,10.5,True,138.6
5,Piraat,2.5,33,Ambrée,10.5,True,138.6
6,Bush 12 Fût,2.2,25,Ambrée,12,True,136.363636
7,Chimay rouge,2.2,33,Trappiste,9,True,135.0
8,Corne du bois des pendus quadruple,3,33,Ambrée,12,True,132.0
9,Queue de charrue triple,2.3,33,Blonde,9,True,129.130435


# Top 20, blonde beers

In [130]:
blonde_beers = display_top(df[(df["Type"] == "Blonde") & (df["Volume"] < 75)])
blonde_beers.to_csv(f"csv/{YEAR}/blonde_ranker.csv")
blonde_beers

,Name,Price,Volume,Type,Degree,Available,Ratio
Rank,,,,,,,
1,Kerel Kaishaku Fût,2.5,25,Blonde,15,True,150.0
2,Kerel Kaishaku,3.5,33,Blonde,15,True,141.428571
3,Kasteel triple,2.6,33,Blonde,11,True,139.615385
4,Bush 10,2.5,33,Blonde,10.5,True,138.6
5,Queue de charrue triple,2.3,33,Blonde,9,True,129.130435
6,Corne du bois des pendus 10 triple,2.8,33,Blonde,10,True,117.857143
7,Duvel,2.4,33,Blonde,8.5,True,116.875
8,Triple plaisir (la),2.3,33,Blonde,8,True,114.782609
9,Carolus triple d'or,2.6,33,Blonde,9,True,114.230769


# Top 20, ambrées

In [131]:
amber_beers = display_top(df[(df["Type"] == "Ambrée") & (df["Volume"] < 75)])
amber_beers.to_csv(f"csv/{YEAR}/amber_ranker.csv")
amber_beers

,Name,Price,Volume,Type,Degree,Available,Ratio
Rank,,,,,,,
1,Piraat,2.5,33,Ambrée,10.5,True,138.6
2,Bush 12 Fût,2.2,25,Ambrée,12,True,136.363636
3,Corne du bois des pendus quadruple,3,33,Ambrée,12,True,132.0
4,Gulden Draak 9000,2.9,33,Ambrée,10.5,True,119.482759
5,Maredsous 10,3.1,33,Ambrée,10,True,106.451613
6,Bon secours heritage,2.5,33,Ambrée,8,True,105.6
7,Troubadour magma,2.9,33,Ambrée,9,True,102.413793
8,Carolus ambrio,2.6,33,Ambrée,8,True,101.538462
9,Quintine ambrée,2.9,33,Ambrée,8.5,True,96.724138


# Top trappistes

In [132]:
trapist_beers = display_top(df[(df["Type"] == "Trappiste") & (df["Volume"] < 75)])
trapist_beers.to_csv(f"csv/{YEAR}/trapist_ranker.csv")
trapist_beers

,Name,Price,Volume,Type,Degree,Available,Ratio
Rank,,,,,,,
1,Chimay rouge,2.2,33,Trappiste,9,True,135.0
2,Westmalle triple,2.5,33,Trappiste,9.5,True,125.4
3,Rochefort triple extra,2.4,33,Trappiste,9,True,123.75
4,Rochefort 10,3.1,33,Trappiste,11.3,True,120.290323
5,Rochefort 8,2.7,33,Trappiste,9.2,True,112.444444
6,Chimay blanche,2.4,33,Trappiste,8,True,110.0
7,Chimay bleue,3,33,Trappiste,9,True,99.0
8,Rochefort 6,2.6,33,Trappiste,7.5,True,95.192308
9,Chimay Verte (150),3.5,33,Trappiste,10,True,94.285714


# Top 20, brunes

In [133]:
brown_beers = display_top(df[(df["Type"] == "Brune") & (df["Volume"] < 75)])
brown_beers.to_csv(f"csv/{YEAR}/brown_ranker.csv")
brown_beers

,Name,Price,Volume,Type,Degree,Available,Ratio
Rank,,,,,,,
1,Gulden Draak,2.8,33,Brune,10.7,True,126.107143
2,Kasteel donker,2.9,33,Brune,11,True,125.172414
3,Peak grand cru,2.8,33,Brune,10.5,True,123.75
4,Black albert,3.5,33,Brune,13,True,122.571429
5,Straffe Hendrik quadrupel,3.1,33,Brune,11,True,117.096774
6,Moinette brune,2.4,33,Brune,8.5,True,116.875
7,Malheur 12,3.4,33,Brune,12,True,116.470588
8,Saint Bernardus Prior 8,2.3,33,Brune,8,True,114.782609
9,Gauloise brune,2.4,33,Brune,8.1,True,111.375


# Top 20, fruitées

In [134]:
fruit_beers = display_top(df[(df["Type"] == "Fruitée") & (df["Volume"] < 75)])
fruit_beers.to_csv(f"csv/{YEAR}/fruit_ranker.csv")
fruit_beers

,Name,Price,Volume,Type,Degree,Available,Ratio
Rank,,,,,,,
1,Queue de charrue rouge,2.7,33,Fruitée,8.7,True,106.333333
2,Gauloise fruits rouges,2.6,33,Fruitée,8.2,True,104.076923
3,Frambush,2.8,33,Fruitée,8.5,True,100.178571
4,Pêche Mel Bush Fût,2,25,Fruitée,8,True,100.0
5,Chouffe cherry Fût,2,25,Fruitée,8,True,100.0
6,Delirium red Fût,2,25,Fruitée,8,True,100.0
7,Val dieu fruitee,3.2,33,Fruitée,9,True,92.8125
8,Barbar Rouge,2.9,33,Fruitée,8,True,91.034483
9,Kasteel red Fût,2.2,25,Fruitée,8,True,90.909091


# Top blanches

In [135]:
white_beers = display_top(df[(df["Type"] == "Blanche") & (df["Volume"] < 75)])
white_beers.to_csv(f"csv/{YEAR}/white_ranker.csv")
white_beers

,Name,Price,Volume,Type,Degree,Available,Ratio
Rank,,,,,,,
1,Bon secours prestige,2.8,33,Blanche,9,True,106.071429
2,St Hubertus Blanche Fût,2,25,Blanche,7.2,True,90.0
3,Chouffe blanche,2.2,33,Blanche,6,True,90.0
4,Blanche de Bruxelles,2,33,Blanche,4.5,True,74.25
5,St bernardus witbier,2.5,33,Blanche,5.5,True,72.6
6,Boriner vice,2.9,33,Blanche,6,True,68.275862
7,Blanche de Namur,1.8,25,Blanche,4.5,True,62.5
8,Troublette,3,33,Blanche,5.6,True,61.6


# Top 75cl

In [136]:
big = display_top(df[df["Volume"]  == 75])
big.to_csv(f"csv/{YEAR}/75cl_ranker.csv")
big

,Name,Price,Volume,Type,Degree,Available,Ratio
Rank,,,,,,,
1,Bush 12,7,75,Ambrée,12,True,128.571429
2,Moinette brune,5,75,Brune,8.5,True,127.5
3,Lupulus Hibernatus,5.7,75,Brune,9,True,118.421053
4,Lupulus blonde,5.5,75,Blonde,8.5,True,115.909091
5,Lupulus brune,5.7,75,Brune,8.5,True,111.842105
6,Binchoise brune,5.3,75,Brune,7.7,True,108.962264
7,Chimay Blanche (Cinq Cents),5.6,75,Trappiste,8,True,107.142857
8,Moinette blonde,6,75,Blonde,8.5,True,106.25
9,Lupulus Organicus,6,75,Blonde,8.5,True,106.25


# Top 50 du rat (à plus que 5° quand même (big up à Hunter))

In [137]:
rat = display_top(df[(df["Degree"] >= 5) & (df["Volume"] < 75) & (df["Price"] <= 2.5)], n=50)
rat = rat.sort_values(by=["Ratio", "Volume"], ascending=[False, True])
rat

,Name,Price,Volume,Type,Degree,Available,Ratio
Rank,,,,,,,
1,Kerel Kaishaku Fût,2.5,25,Blonde,15,True,150.0
2,Bush 10,2.5,33,Blonde,10.5,True,138.6
3,Piraat,2.5,33,Ambrée,10.5,True,138.6
4,Bush 12 Fût,2.2,25,Ambrée,12,True,136.363636
5,Chimay rouge,2.2,33,Trappiste,9,True,135.0
6,Queue de charrue triple,2.3,33,Blonde,9,True,129.130435
7,Westmalle triple,2.5,33,Trappiste,9.5,True,125.4
8,Rochefort triple extra,2.4,33,Trappiste,9,True,123.75
9,Duvel,2.4,33,Blonde,8.5,True,116.875
